In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cars-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4579885%2F7817411%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240317%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240317T052621Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6f5b21c954b9d687ef88164753dda4221e75560258e5ac825a78352483da192dad6f45f563dfb48d8336f5f7bebed419c0d4d2992425fe4253d461923b0e6ad5f9584e8e83289e791b6feed861608d0fa9ebcbcceaeeee6ceeaa87ad7cb0e462fa18b9ecd656cd93858876851bfd4aabdfcd649b5fb06da489fa7a051a529215704db4efe54f8600440bc3024e3819624cf344648736cb29f5c7b5bae88eedf454e5386444389ee475070989c91a6cd0b77013dc0d123340938c7c48e4be0efc3a1fbffe4df346a5c6e911d59cd4223d4ae6af8a1b684529fa3cf40cf9282698bcf8cdea6970569d943d8de60417e53d3a5bbd8c400f4916198e711b3e1947f3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 173037 bytes downloaded
Downloaded and uncompressed: cars-dataset
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cars-dataset/used_cars_data.csv


In [4]:
df=pd.read_csv('/kaggle/input/cars-dataset/used_cars_data.csv')

In [5]:
df.head(3)

,S.No.,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50


In [6]:
filtre=df['Name'].str.contains("Civic")
df[filtre]

,S.No.,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
114,114,Honda Civic 2006-2010 1.8 V MT,Delhi,2010,71088,Petrol,Manual,First,13.5 kmpl,1799 CC,130 bhp,5.0,NaN,3.35
139,139,Honda Civic 2006-2010 1.8 V MT,Delhi,2012,88578,Petrol,Manual,First,13.5 kmpl,1799 CC,130 bhp,5.0,NaN,3.95
447,447,Honda Civic 2006-2010 1.8 S AT,Pune,2008,61938,Petrol,Automatic,Second,12.9 kmpl,1799 CC,130 bhp,5.0,NaN,2.65
554,554,Honda Civic 2006-2010 1.8 S MT,Ahmedabad,2007,51000,Petrol,Manual,First,13.5 kmpl,1799 CC,130 bhp,5.0,NaN,4.90
978,978,Honda Civic 2010-2013 1.8 V MT,Kolkata,2010,38000,Petrol,Manual,First,14.8 kmpl,1799 CC,130.2 bhp,5.0,NaN,2.00
1174,1174,Honda Civic 2006-2010 1.8 V MT,Delhi,2008,72000,Petrol,Manual,First,13.5 kmpl,1799 CC,130 bhp,5.0,NaN,2.50
1201,1201,Honda Civic 2010-2013 1.8 S MT,Mumbai,2012,92000,Petrol,Manual,Second,14.8 kmpl,1799 CC,130.2 bhp,5.0,NaN,3.35
1278,1278,Honda Civic 2010-2013 1.8 S MT,Mumbai,2010,86000,Petrol,Manual,First,14.8 kmpl,1799 CC,130.2 bhp,5.0,NaN,2.90
1507,1507,Honda Civic 2006-2010 1.8 V AT,Hyderabad,2008,95000,Petrol,Automatic,Second,12.9 kmpl,1799 CC,130 bhp,5.0,NaN,3.00
1751,1751,Honda Civic 2006-2010 1.8 S MT,Chennai,2006,110000,Petrol,Manual,First,13.5 kmpl,1799 CC,130 bhp,5.0,NaN,2.65


In [9]:
def searchBrand(query):
    query=query.title()
    df=pd.read_csv('/kaggle/input/cars-dataset/used_cars_data.csv')
    filtre=df['Name'].str.contains(query)
    return df[filtre]

In [8]:
searchBrand('vitara')

,S.No.,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
36,36,Maruti Vitara Brezza ZDi Plus,Kochi,2018,50075,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,11.12 Lakh,8.63
119,119,Maruti Vitara Brezza VDi,Mumbai,2016,55000,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,9.96 Lakh,7.50
201,201,Maruti Vitara Brezza ZDi Plus Dual Tone,Mumbai,2016,20000,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,11.98 Lakh,9.00
343,343,Maruti Vitara Brezza LDi,Chennai,2016,45000,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,9.12 Lakh,7.15
372,372,Maruti Vitara Brezza ZDi Plus,Hyderabad,2017,39000,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,12 Lakh,9.49
618,618,Maruti Vitara Brezza ZDi,Pune,2016,21558,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,10.65 Lakh,8.90
668,668,Maruti Vitara Brezza ZDi Plus Dual Tone,Coimbatore,2017,18108,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,12.56 Lakh,11.31
688,688,Maruti Vitara Brezza LDi Option,Pune,2017,38000,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,NaN,7.99
885,885,Maruti Grand Vitara AT,Chennai,2008,77000,Petrol,Automatic,Fourth & Above,9.7 kmpl,1995 CC,163.5 bhp,5.0,NaN,5.50
1077,1077,Maruti Vitara Brezza ZDi Plus,Kochi,2019,34688,Diesel,Manual,First,24.3 kmpl,1248 CC,88.5 bhp,5.0,11.2 Lakh,10.49
